# Feature Transformation with Amazon a SageMaker Processing Job and Scikit-Learn

In this notebook, we convert raw text into BERT embeddings.  This will allow us to perform natural language processing tasks such as text classification.

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Scikit-Learn are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Scikit-Learn in a managed SageMaker environment to run our processing workload.

# NOTE:  THIS NOTEBOOK WILL TAKE A 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)


## Contents

1. Setup Environment
1. Setup Input Data
1. Setup Output Data
1. Build a Spark container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Data

# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
%store -r training_job_name

In [ ]:
try:
    training_job_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(training_job_name)

In [ ]:
%store -r raw_input_data_s3_uri

In [ ]:
try:
    raw_input_data_s3_uri
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the PREPARE section before you continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
print(raw_input_data_s3_uri)

In [ ]:
%store -r max_seq_length

In [ ]:
try:
    max_seq_length
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(max_seq_length)

In [ ]:
%store -r experiment_name

In [ ]:
try:
    experiment_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(experiment_name)

In [ ]:
%store -r trial_name

In [ ]:
try:
    trial_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(trial_name)

In [ ]:
from sagemaker.tensorflow.estimator import TensorFlow

describe_training_job_response = sm.describe_training_job(TrainingJobName=training_job_name)
print(describe_training_job_response)

In [ ]:
model_dir_s3_uri = describe_training_job_response['ModelArtifacts']['S3ModelArtifacts'].replace('model.tar.gz', '')
model_dir_s3_uri

# Run the Processing Job using Amazon SageMaker

Next, use the Amazon SageMaker Python SDK to submit a processing job using our custom python script.

# Create the `Experiment Config`

In [ ]:
experiment_config = {
    'ExperimentName': experiment_name,
    'TrialName': trial_name,
    'TrialComponentDisplayName': 'evaluate'
}

# Set the Processing Job Hyper-Parameters 

In [ ]:
processing_instance_type='ml.m5.xlarge'
processing_instance_count=1

# Choosing a `max_seq_length` for BERT
Since a smaller `max_seq_length` leads to faster training and lower resource utilization, we want to find the smallest review length that captures `70%` of our reviews.

Remember our distribution of review lengths from a previous section?

```
mean         67.930174
std         130.954079
min           1.000000
10%           4.000000
20%          14.000000
30%          21.000000
40%          25.000000
50%          31.000000
60%          42.000000
70%          59.000000
80%          87.000000
90%         149.000000
100%       5347.000000
max        5347.000000
```

![](img/review_word_count_distribution.png)

Review length `59` represents the `70th` percentile for this dataset.  However, it's best to stick with powers-of-2 when using BERT.  So let's choose `64` as this is the smallest power-of-2 greater than `59`.  Reviews with length > `64` will be truncated to `64`.

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(framework_version='0.23-1',
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=processing_instance_count,
                             max_runtime_in_seconds=7200)

# Review the Processing Script

In [ ]:
!pygmentize evaluate_model_metrics.py

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor.run(code='evaluate_model_metrics.py',
              inputs=[
                  ProcessingInput(input_name='model-tar-s3-uri',
                                  source=model_dir_s3_uri,
                                  destination='/opt/ml/processing/input/model/'),
                  ProcessingInput(input_name='evaluation-data-s3-uri',
                                  source=raw_input_data_s3_uri,
                                  destination='/opt/ml/processing/input/data/')
              ],
              outputs=[
                  ProcessingOutput(s3_upload_mode='EndOfJob',
                                   output_name='metrics',
                                   source='/opt/ml/processing/output/metrics'),
              ],
              arguments=[
                  '--max-seq-length', str(max_seq_length)
              ],
              experiment_config=experiment_config,
              logs=True,
              wait=False)

In [ ]:
scikit_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']
print(scikit_processing_job_name)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, scikit_processing_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, scikit_processing_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(bucket, scikit_processing_job_name, region)))


# Monitor the Processing Job

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=scikit_processing_job_name,
                                                                            sagemaker_session=sess)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
processing_evaluation_metrics_job_name = processing_job_description['ProcessingJobName']
print(processing_evaluation_metrics_job_name)

In [ ]:
running_processor.wait(logs=False)

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output Data

Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [ ]:
processing_job_description = running_processor.describe()

output_config = processing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'metrics':
        processed_metrics_s3_uri = output['S3Output']['S3Uri']
        
print(processed_metrics_s3_uri)

In [ ]:
!aws s3 ls $processed_metrics_s3_uri/

# Pass Variables to the Next Notebook(s)

In [ ]:
%store processing_evaluation_metrics_job_name

In [ ]:
%store processed_metrics_s3_uri

In [ ]:
%store

# Show the Experiment Tracking Lineage

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

import pandas as pd
pd.set_option("max_colwidth", 500)
#pd.set_option("max_rows", 100)

experiment_analytics = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    sort_by="CreationTime",
    sort_order="Descending"
)

experiment_analytics_df = experiment_analytics.dataframe()
experiment_analytics_df

In [ ]:
trial_component_name=experiment_analytics_df.TrialComponentName[0]
print(trial_component_name)

In [ ]:
trial_component_description=sm.describe_trial_component(TrialComponentName=trial_component_name)
trial_component_description

In [ ]:
from sagemaker.lineage.visualizer import LineageTableVisualizer

lineage_table_viz = LineageTableVisualizer(sess)
lineage_table_viz_df = lineage_table_viz.show(processing_job_name=processing_evaluation_metrics_job_name)
lineage_table_viz_df

In [ ]:
# from sagemaker.lineage.visualizer import LineageTableVisualizer

# lineage_table_viz = LineageTableVisualizer(sess)
# lineage_table_viz_df = lineage_table_viz.show(trial_component_name=trial_component_name)
# lineage_table_viz_df

In [ ]:
from sagemaker.analytics import ArtifactAnalytics

artifact_analytics = ArtifactAnalytics()

artifacts_df = artifact_analytics.dataframe()
artifacts_df

In [ ]:
# from sagemaker.lineage.association import Association
# from sagemaker.lineage.artifact import Artifact

# incoming_associations = Association.list(destination_arn='arn:aws:sagemaker:us-east-1:835319576252:artifact/ebfc31f9f6007feb1a70daaf24339c08',
#                                 sort_by='CreationTime', 
#                                 sort_order='Descending')

# for association in enumerate(incoming_associations):
#     print(association)

In [ ]:
# from sagemaker.lineage.association import Association
# from sagemaker.lineage.artifact import Artifact

# incoming_associations = Association.list(source_arn='arn:aws:sagemaker:us-east-1:835319576252:artifact/ebfc31f9f6007feb1a70daaf24339c08',
#                                 sort_by='CreationTime', 
#                                 sort_order='Descending')

# for association in enumerate(incoming_associations):
#     print(association)

In [ ]:
# # list all the contexts
# from sagemaker.lineage.context import Context

# contexts = Context.list(sort_by='CreationTime', 
#                         sort_order='Descending')

# for ctx in contexts:
#     print(ctx.source.source_uri)

In [ ]:
# from sagemaker.lineage.action import Action

# actions = Action.list(sort_by='CreationTime', sort_order='Descending')

# for action in actions:
#     print(action)

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();